In [3]:
import requests
import os
import boto3
from dotenv import load_dotenv
from github_python_fetch import fetch_function
from query_package import get_query_package

# GitHub Access 
token = os.getenv('GITHUBTOKEN')

# AWS Access
access = os.getenv('ACCESS')
secret = os.getenv('SECRET')

# RDS Access
username = os.getenv('USER')
password = os.getenv('PASSWORD')
server = os.getenv('SERVER')
db = os.getenv('DB')


# Access RDS Functions via GitHub
rds_functions_url = 'https://raw.githubusercontent.com/CharlesRoss-MBakerIntl/Tidal-Basin-Functions/main/rds_connector.py' # Set url to python file of github
rds_functions = fetch_function(rds_functions_url, token) # Pull function from github using requests
exec(rds_functions) # Execute the file


# Pull Query Package from File
query_package = get_query_package()


# Store RDS Functions Locally
#conn, cursor = rds_connection(username, password, db, server) # Connect to RDS Database
#rds = RDSTablePull(conn, cursor, query_package) # Create Instance of RDS Table

In [ ]:
for item in rds.cleaning_versions:
    
    # Create File Name
    if item['Field'] == None:
        file_name = f"{item['Step']}"
    else:
        file_name = f"{item['Field']}: {item['Step']}"

    # Store DataFrame as CSV
    #data = item['Result'].to_csv(f"{file_name}.csv")

In [5]:
# Bucket and file information
bucket_name = 'reporting-external'

# Create a S3 client
s3 = boto3.client('s3', aws_access_key_id=access, aws_secret_access_key=secret)

result = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied